In [3]:
import os
import rasterio
from rasterio.merge import merge
from tqdm import tqdm

# 定义输出目录
output_dir = 'G:/Hangkai/Global forest edge/2000Edge_classified/'
output_dir_merge = 'G:/Hangkai/Global forest edge/'

# 获取所有输出文件的路径列表
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.tif')]

# 直接使用merge工具合并所有图像
# 注意：这里不再先读取所有数据，而是直接提供文件路径给merge函数
mosaic, out_trans = merge(all_files, indexes=1, dtype='int8')

# 更新metadata
with rasterio.open(all_files[0]) as src0:
    meta = src0.meta

meta.update({"driver": "GTiff",
             "height": mosaic.shape[1],
             "width": mosaic.shape[2],
             "transform": out_trans,
             "dtype": 'int8',
             "nodata": None
            })

# 保存拼接后的文件
mosaic_output_path = os.path.join(output_dir_merge, 'Mosaic_Band_1.tif')
with rasterio.open(mosaic_output_path, 'w', **meta) as dst:
    dst.write(mosaic.astype(rasterio.int8), 1)

MemoryError: Unable to allocate 751. GiB for an array with shape (1, 560000, 1440000) and data type int8

In [ ]:
import os
import rasterio
from netCDF4 import Dataset
import numpy as np
from tqdm import tqdm

# 定义输出目录
output_dir = 'G:/Hangkai/Global forest edge/2000Edge_classified/'
output_dir_merge = 'G:/Hangkai/Global forest edge/'

# 获取所有输出文件的路径列表
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.tif')]

# 使用第一个文件获取尺寸，以创建netCDF维度
with rasterio.open(all_files[0]) as src:
    height, width = src.shape

# 创建一个新的netCDF文件并初始化所有值为0
mosaic_output_path = os.path.join(output_dir_merge, 'Mosaic.nc')
rootgrp = Dataset(mosaic_output_path, "w", format="NETCDF4")

rootgrp.createDimension("height", height)
rootgrp.createDimension("width", width)

# 创建变量并启用压缩
var_1 = rootgrp.createVariable("Forest edge changing type", 'i1', ("height", "width"), zlib=True)
var_2 = rootgrp.createVariable("Forest edge difference", 'f4', ("height", "width"), zlib=True)

var_1[:] = np.zeros((height, width), dtype=np.int8)
var_2[:] = np.zeros((height, width), dtype=np.float32)

# 逐个读取TIFF文件并更新netCDF文件
for file in tqdm(all_files):
    with rasterio.open(file) as src:
        data_1 = src.read(1).astype(np.int8)  # 读取第一个波段并调整数据类型
        data_2 = src.read(2).astype(np.float32)  # 读取第二个波段并调整数据类型

        var_1[:] += data_1
        var_2[:] += data_2

# 关闭netCDF文件
rootgrp.close()

 49%|█████████████████████████████████████                                      | 129/261 [7:29:19<7:39:35, 208.91s/it]

In [ ]:
import os
import rasterio
from netCDF4 import Dataset
import numpy as np

output_dir = '/mnt/cephfs/scratch/groups/chen_group/hangkai/2000Edge_classified/'
output_dir_merge = '/mnt/cephfs/scratch/groups/chen_group/hangkai/'

all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.tif')]

with rasterio.open(all_files[0]) as src:
    height, width = src.shape

mosaic_output_path = os.path.join(output_dir_merge, '01Class.nc')
rootgrp = Dataset(mosaic_output_path, "w", format="NETCDF4")

rootgrp.createDimension("height", height)
rootgrp.createDimension("width", width)

var_1 = rootgrp.createVariable("Forest edge changing type", 'i1', ("height", "width"), zlib=True)

var_1[:] = np.zeros((height, width), dtype=np.int8)

for file in all_files:
    with rasterio.open(file) as src:
        data_1 = src.read(1).astype(np.int8)

        var_1[:] += data_1

rootgrp.close()

In [ ]:
import os
import rasterio
from rasterio.merge import merge
from tqdm import tqdm

# 定义输出目录
output_dir = 'G:/Hangkai/Global forest edge/2000Edge_classified/'
output_dir_merge = 'G:/Hangkai/Global forest edge/'

# 获取所有输出文件的路径列表
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.tif')]

# 直接使用merge工具合并所有图像
# 注意：这里不再先读取所有数据，而是直接提供文件路径给merge函数
mosaic, out_trans = merge(all_files, indexes=1, dtype='int8')

# 更新metadata
with rasterio.open(all_files[0]) as src0:
    meta = src0.meta

meta.update({"driver": "GTiff",
             "height": mosaic.shape[1],
             "width": mosaic.shape[2],
             "transform": out_trans,
             "dtype": 'int8',
             "nodata": None
            })

# 保存拼接后的文件
mosaic_output_path = os.path.join(output_dir_merge, 'Mosaic_Band_1.tif')
with rasterio.open(mosaic_output_path, 'w', **meta) as dst:
    dst.write(mosaic.astype(rasterio.int8), 1)

In [ ]:
import os
import rasterio
import numpy as np
output_dir_class = 'G:/Hangkai/Global forest edge/2000Edge_classified_01_class/'
output_dir_diff = 'G:/Hangkai/Global forest edge/2000Edge_classified_01_diff/'

output_dir = output_dir_diff

# 初始化变量来保存最大和最小值
max_value_band1 = float('-inf')
min_value_band1 = float('inf')

# 遍历所有文件
for file_name in os.listdir(output_dir):
    with rasterio.open(os.path.join(output_dir, file_name)) as src:
        # 读取band1和band2
        band1 = src.read(1)

        # 更新最大和最小值
        max_value_band1 = max(max_value_band1, band1.max())
        min_value_band1 = min(min_value_band1, band1.min())


        print(f"Band 1: Max Value = {max_value_band1}, Min Value = {min_value_band1}")


Band 1: Max Value = 77.5802993774414, Min Value = -213.55929565429688
